# OpenSearch MCP Server - Complete Feature Demonstration

This notebook demonstrates all features of the **opensearch-mcp-server-py** package, which provides a Model Context Protocol (MCP) bridge between AI agents and OpenSearch clusters.

## Learning Objectives
- Understand MCP (Model Context Protocol) architecture
- Install and configure opensearch-mcp-server
- Integrate MCP server with LangChain and OpenAI
- Use all available MCP tools for OpenSearch operations
- Execute natural language queries against OpenSearch data

## Architecture Overview

```mermaid
graph TB
    subgraph "User Layer"
        A[User Natural Language Query]
    end
    
    subgraph "AI Agent Layer"
        B[LangChain Agent]
        C[OpenAI LLM GPT-4]
    end
    
    subgraph "MCP Layer"
        D[MCP Client]
        E[MCP Server<br/>opensearch-mcp-server-py]
        F[MCP Tools<br/>15+ Operations]
    end
    
    subgraph "Data Layer"
        G[OpenSearch Cluster]
        H[(Ecommerce Index)]
    end
    
    A -->|Natural Language| B
    B <-->|Tool Calling| C
    C <-->|Function Calls| D
    D <-->|SSE/HTTP Stream| E
    E -->|Executes| F
    F <-->|REST API| G
    G <-->|CRUD| H
    
    style A fill:#e1f5ff
    style B fill:#fff9e6
    style C fill:#fff9e6
    style E fill:#e8f5e9
    style F fill:#e8f5e9
    style G fill:#fce4ec
    style H fill:#fce4ec
```

### Flow Explanation
1. **User Layer**: User asks questions in natural language
2. **AI Agent Layer**: LangChain orchestrates the conversation using OpenAI's GPT-4 for reasoning
3. **MCP Layer**: MCP client/server handle tool discovery and execution
4. **Data Layer**: OpenSearch cluster stores and retrieves data

## 1. Installation and Setup

### What is MCP?
**Model Context Protocol (MCP)** is an open protocol that standardizes how AI applications provide context to Large Language Models (LLMs). It allows LLMs to securely access data and tools through a unified interface.

### Key Components
- **MCP Server**: Exposes tools and data sources
- **MCP Client**: Connects to servers and makes tools available to LLMs
- **Tools**: Specific operations (search, index, delete, etc.)
- **Transport**: Communication protocol (SSE/HTTP streaming)

In [3]:
# Install required packages
!uv add opensearch-mcp-server-py==0.4.0
!uv add langchain==0.3.27 langchain-mcp-adapters==0.1.12 langchain-openai==0.3.33

Resolved 290 packages in 3ms
Audited 284 packages in 1ms
Resolved 290 packages in 3ms
Audited 284 packages in 1ms
Resolved 290 packages in 3ms
Audited 284 packages in 1ms


## 2. Environment Configuration

### Configuration Parameters Explained
- **OPENSEARCH_URL**: Endpoint of your OpenSearch cluster
- **OPENSEARCH_USERNAME/PASSWORD**: Basic authentication credentials
- **OPENSEARCH_SSL_VERIFY**: Whether to verify SSL certificates (set to false for local dev)
- **OPENAI_API_KEY**: Your OpenAI API key for GPT-4 access

In [1]:
import os
import json
import asyncio
from getpass import getpass

# OpenSearch Configuration
os.environ['OPENSEARCH_URL'] = 'https://localhost:9200'
os.environ['OPENSEARCH_USERNAME'] = 'admin'
os.environ['OPENSEARCH_PASSWORD'] = 'Developer@123'
os.environ['OPENSEARCH_SSL_VERIFY'] = 'false'

# OpenAI Configuration
if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API Key: ')

print("✓ Environment configured successfully")

✓ Environment configured successfully


## 3. Create and Populate Ecommerce Index

### Index Mappings
**Mappings** define the structure of documents in an index, specifying:
- **Field types**: text, keyword, date, numeric, etc.
- **Analyzers**: How text is processed for searching
- **Multi-fields**: Store data in multiple ways (e.g., both text and keyword)

In [2]:
from opensearchpy import OpenSearch
from opensearchpy.helpers import bulk
import warnings
warnings.filterwarnings('ignore')

# Initialize OpenSearch client
client = OpenSearch(
    hosts=[{'host': 'localhost', 'port': 9200}],
    http_auth=('admin', 'Developer@123'),
    use_ssl=True,
    verify_certs=False,
    ssl_show_warn=False
)

print("✓ Connected to OpenSearch cluster")
print(f"Cluster info: {client.info()['version']['distribution']} {client.info()['version']['number']}")

✓ Connected to OpenSearch cluster
Cluster info: opensearch 3.3.0


In [3]:
# Load index mappings
with open('../../0. DATA/ecommerce-field_mappings.json', 'r') as f:
    index_mappings = json.load(f)

# Delete existing index if it exists
if client.indices.exists(index='ecommerce'):
    client.indices.delete(index='ecommerce')
    print("✓ Deleted existing ecommerce index")

# Create the index with mappings
client.indices.create(index='ecommerce', body=index_mappings)
print("✓ Created ecommerce index with mappings")

# Display sample mapping structure
print("\nSample field mappings:")
print(json.dumps({
    'customer_first_name': index_mappings['mappings']['properties']['customer_first_name'],
    'order_date': index_mappings['mappings']['properties']['order_date']
}, indent=2))

✓ Created ecommerce index with mappings

Sample field mappings:
{
  "customer_first_name": {
    "type": "text",
    "fields": {
      "keyword": {
        "type": "keyword",
        "ignore_above": 256
      }
    }
  },
  "order_date": {
    "type": "date"
  }
}


In [4]:
# Load and index ecommerce data
data = []
with open('../../0. DATA/ecommerce.json', 'r') as file:
    for line in file:
        data.append(line.strip() + '\n')

# Bulk index the data
bulk(client, actions=data, index='ecommerce', refresh=True)
print(f"✓ Indexed {len(data)} documents into ecommerce index")

# Verify indexing
count = client.count(index='ecommerce')['count']
print(f"Total documents in ecommerce index: {count}")

✓ Indexed 9350 documents into ecommerce index
Total documents in ecommerce index: 9350


## 4. Start MCP Server and Initialize LangChain Client

### MCP Server Modes
- **Single Mode**: Connect to one OpenSearch cluster (we use this)
- **Multi Mode**: Connect to multiple clusters with different configurations

### Transport Types
- **STDIO**: Standard input/output (for CLI tools)
- **Stream (SSE)**: Server-Sent Events over HTTP (for programmatic access) ← We use this

In [5]:
import subprocess
import time
import requests

# Start MCP server in streaming mode (background process)
mcp_process = subprocess.Popen(
    ['python', '-m', 'mcp_server_opensearch', '--transport', 'stream', '--port', '9900'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    env=os.environ.copy()
)

print("Starting MCP server...")
time.sleep(5)  # Wait for server to start

# Verify server is running
try:
    response = requests.get('http://localhost:9900/health', timeout=5)
    print("✓ MCP server started successfully on port 9900")
except:
    print("⚠ Server may still be starting...")

# Store process ID for cleanup
print(f"MCP Server PID: {mcp_process.pid}")

Starting MCP server...
✓ MCP server started successfully on port 9900
MCP Server PID: 11420


### Initialize LangChain MCP Client

The **MultiServerMCPClient** connects to our MCP server and exposes its tools to LangChain.
The **ChatOpenAI** model processes natural language and decides which tools to use.

In [6]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType, initialize_agent
from dotenv import load_dotenv

load_dotenv("../../.env")

# Initialize OpenAI LLM
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0  # Deterministic responses
)

print("✓ Initialized OpenAI GPT-4o model")

✓ Initialized OpenAI GPT-4o model


### Create Agent Helper Function

This function creates a LangChain agent that:
1. Connects to the MCP server
2. Retrieves available tools
3. Initializes an agent with OpenAI function calling
4. Executes queries and returns results

In [7]:
async def query_opensearch(question: str, verbose: bool = True):
    """
    Query OpenSearch using natural language through MCP and LangChain.
    
    Args:
        question: Natural language query
        verbose: Show agent's reasoning process
    
    Returns:
        Agent's response
    """
    # Initialize MCP client (new API pattern)
    client = MultiServerMCPClient({
        "opensearch-mcp-server": {
            "transport": "sse",
            "url": "http://localhost:9900/sse",
        }
    })
    
    # Get tools from the client
    tools = await client.get_tools()
    
    # Initialize agent with tools
    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.OPENAI_FUNCTIONS,
        verbose=verbose,
    )
    
    # Execute query
    result = await agent.ainvoke({"input": question})
    return result['output']

print("✓ Query helper function defined")

✓ Query helper function defined


## 5. Index Management Operations

```mermaid
graph LR
    A[User Query:<br/>List all indices] --> B[LangChain Agent]
    B --> C[MCP Tool:<br/>list_indices]
    C --> D[OpenSearch API:<br/>GET _cat/indices]
    D --> E[Returns:<br/>Index list]
    E --> B
    B --> F[Natural Language<br/>Response]
    
    style A fill:#e1f5ff
    style C fill:#e8f5e9
    style D fill:#fce4ec
    style F fill:#fff9e6
```

### Available Tools
- **list_indices**: List all indices in cluster
- **get_index**: Get detailed information about an index
- **create_index**: Create a new index with mappings
- **delete_index**: Delete an existing index

### Query 1: List All Indices

In [14]:
result = await query_opensearch("List all indices in the OpenSearch cluster")
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)

/tmp/ipykernel_7833/282870703.py:24: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...

Invoking: `ListIndexTool` with `{'include_detail': False}`


Indices:
[
  ".plugins-ml-config",
  "top_queries-2025.11.11-85601",
  "ecommerce",
  "security-auditlog-2025.11.11",
  ".opendistro_security",
  ".kibana_1"
]
Invoking: `ListIndexTool` with `{'include_detail': False}`


Indices:
[
  ".plugins-ml-config",
  "top_queries-2025.11.11-85601",
  "ecommerce",
  "security-auditlog-2025.11.11",
  ".opendistro_security",
  ".kibana_1"
]Here are the indices in the OpenSearch cluster:

1. `.plugins-ml-config`
2. `top_queries-2025.11.11-85601`
3. `ecommerce`
4. `security-auditlog-2025.11.11`
5. `.opendistro_security`
6. `.kibana_1`

> Finished chain.

RESULT:
Here are the indices in the OpenSearch cluster:

1. `.plugins-ml-config`
2. `top_queries-2025.11.11-85601`
3. `ecommerce`
4. `security-auditlog-2025.11.11`
5. `.opendistro_security`
6. `.kibana_1`
Here are the indices in the OpenSearch cluster:

1. `.plugins-ml-config`
2. `top_queries-2025.11

### Query 2: Get Index Details

**Index details** include:
- **Mappings**: Field types and configurations
- **Settings**: Shards, replicas, analyzers
- **Aliases**: Alternative names for the index

In [15]:
result = await query_opensearch("Show me detailed information about the ecommerce index including mappings and settings")
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...

Invoking: `ListIndexTool` with `{'index': 'ecommerce', 'include_detail': True}`


Index information for ecommerce:
{
  "ecommerce": {
    "aliases": {},
    "mappings": {
      "properties": {
        "category": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword"
            }
          }
        },
        "currency": {
          "type": "keyword"
        },
        "customer_birth_date": {
          "type": "date"
        },
        "customer_first_name": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256
            }
          }
        },
        "customer_full_name": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256
            }
          }
        },
        "customer_gender": {
          "type": "keywo

### Query 3: Create a New Index
- Not yet available

In [16]:
result = await query_opensearch(
    """Create a new index called 'test_products' with the following structure:
    - product_name: text field
    - price: float
    - category: keyword
    - created_date: date
    Set number of shards to 1 and replicas to 0"""
)
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...

Invoking: `ListIndexTool` with `{'index': 'test_products'}`


Error listing indices: NotFoundError(404, 'index_not_found_exception', 'no such index [test_products]', test_products, index_or_alias)
Invoking: `ListIndexTool` with `{'index': 'test_products'}`


Error listing indices: NotFoundError(404, 'index_not_found_exception', 'no such index [test_products]', test_products, index_or_alias)
Invoking: `IndexMappingTool` with `{'index': 'test_products'}`


Error getting mapping: NotFoundError(404, 'index_not_found_exception', 'no such index [test_products]', test_products, index_or_alias)
Invoking: `IndexMappingTool` with `{'index': 'test_products'}`


Error getting mapping: NotFoundError(404, 'index_not_found_exception', 'no such index [test_products]', test_products, index_or_alias)
Invoking: `ListIndexTool` with `{'include_detail': True}`
responded: It seems that the index 'test_products' does not exist. Let's create it with the specified struc

## 6. Document Operations

```mermaid
graph TB
    A[Document Operations] --> B[Index Document]
    A --> C[Get Document]
    A --> D[Delete Document]
    A --> E[Delete by Query]
    
    B --> F[Adds/Updates<br/>document by ID]
    C --> G[Retrieves<br/>document by ID]
    D --> H[Removes<br/>document by ID]
    E --> I[Removes documents<br/>matching query]
    
    style A fill:#e1f5ff
    style B fill:#e8f5e9
    style C fill:#e8f5e9
    style D fill:#ffe8e8
    style E fill:#ffe8e8
```

### CRUD Operations
- **Create**: Index new documents
- **Read**: Get documents by ID
- **Update**: Index document with same ID (overwrites)
- **Delete**: Remove documents individually or by query

### Query 4: Index a New Document

In [17]:
result = await query_opensearch(
    """Add a new product to the test_products index:
    - product_name: 'Wireless Mouse'
    - price: 29.99
    - category: 'Electronics'
    - created_date: '2025-11-10'
    Use document ID 'prod_001'"""
)
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...

Invoking: `IndexMappingTool` with `{'index': 'test_products'}`


Error getting mapping: NotFoundError(404, 'index_not_found_exception', 'no such index [test_products]', test_products, index_or_alias)
Invoking: `IndexMappingTool` with `{'index': 'test_products'}`


Error getting mapping: NotFoundError(404, 'index_not_found_exception', 'no such index [test_products]', test_products, index_or_alias)
Invoking: `ListIndexTool` with `{'include_detail': True}`


All indices information:
[
  {
    "health": "green",
    "status": "open",
    "index": ".plugins-ml-config",
    "uuid": "a9SaiEfNTQOhFOTqseM2vg",
    "pri": "1",
    "rep": "1",
    "docs.count": "1",
    "docs.deleted": "0",
    "store.size": "8.1kb",
    "pri.store.size": "4kb"
  },
  {
    "health": "green",
    "status": "open",
    "index": "top_queries-2025.11.11-85601",
    "uuid": "XE5xrXJaSl6OLJqClKkkng",
    "pri": "1",
    "rep": "1",
    "docs.count": "13",
    "docs.deleted": "0

### Query 5: Get a Document by ID

In [20]:
result = await query_opensearch("Retrieve the document with \"order_id\":584021 from the ecommerce index")
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...

Invoking: `SearchIndexTool` with `{'index': 'ecommerce', 'query': {'query': {'match': {'order_id': 584021}}}}`


Search results from ecommerce:
{
  "took": 9,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 1,
      "relation": "eq"
    },
    "max_score": 1.0,
    "hits": [
      {
        "_index": "ecommerce",
        "_id": "oZWAcJoBdcTuNTzMEimh",
        "_score": 1.0,
        "_source": {
          "category": [
            "Women's Clothing"
          ],
          "currency": "EUR",
          "customer_first_name": "Mary",
          "customer_full_name": "Mary Bailey",
          "customer_gender": "FEMALE",
          "customer_id": 20,
          "customer_last_name": "Bailey",
          "customer_phone": "",
          "day_of_week": "Sunday",
          "day_of_week_i": 6,
          "email": "mary@bailey-family.zzz",
          "manufa

## 7. Search and Query Operations

```mermaid
graph TB
    A[Search Types] --> B[Match Query]
    A --> C[Term Query]
    A --> D[Range Query]
    A --> E[Bool Query]
    
    B --> F[Full-text search<br/>analyzed text]
    C --> G[Exact match<br/>keywords]
    D --> H[Numeric/Date<br/>ranges]
    E --> I[Combine queries<br/>must/should/must_not]
    
    style A fill:#e1f5ff
    style B fill:#fff9e6
    style C fill:#fff9e6
    style D fill:#fff9e6
    style E fill:#e8f5e9
```

### Query DSL (Domain Specific Language)
OpenSearch uses JSON-based query syntax:
- **match**: Full-text search with analysis
- **term**: Exact match (case-sensitive for keywords)
- **range**: Numeric or date ranges (gte, lte, gt, lt)
- **bool**: Combine queries with logical operators

### Query 9: Simple Match Query

**Match query** analyzes the search text and finds relevant documents.

In [21]:
result = await query_opensearch("Find all customers with first name 'Eddie' in the ecommerce index")
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...

Invoking: `SearchIndexTool` with `{'index': 'ecommerce', 'query': {'query': {'match': {'first_name': 'Eddie'}}}}`


Search results from ecommerce:
{
  "took": 2,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 0,
      "relation": "eq"
    },
    "max_score": null,
    "hits": []
  }
}
Invoking: `SearchIndexTool` with `{'index': 'ecommerce', 'query': {'query': {'match': {'first_name': 'Eddie'}}}}`


Search results from ecommerce:
{
  "took": 2,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 0,
      "relation": "eq"
    },
    "max_score": null,
    "hits": []
  }
}There are no customers with the first name 'Eddie' in the ecommerce index.

> Finished chain.

RESULT:
There are no customers with the first name 'Eddie' in the ecommerce i

### Query 10: Search with Multiple Conditions

**Bool query** structure:
- **must**: All conditions must match (AND)
- **should**: At least one condition should match (OR)
- **must_not**: Conditions must not match (NOT)
- **filter**: Must match but doesn't affect scoring

In [25]:
result = await query_opensearch(
    """Search the ecommerce index for orders where:
    - Customer gender is 'FEMALE'
    - Category contains 'Clothing'
    - order date is in year 2016
    Return only 5 results"""
)
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...

Invoking: `SearchIndexTool` with `{'index': 'ecommerce', 'query': {'size': 5, 'query': {'bool': {'must': [{'term': {'customer_gender': 'FEMALE'}}, {'match': {'category': 'Clothing'}}, {'range': {'order_date': {'gte': '2016-01-01', 'lt': '2017-01-01'}}}]}}}}`


Search results from ecommerce:
{
  "took": 28,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 1964,
      "relation": "eq"
    },
    "max_score": 2.0933888,
    "hits": [
      {
        "_index": "ecommerce",
        "_id": "ZZWAcJoBdcTuNTzMFTPH",
        "_score": 2.0933888,
        "_source": {
          "category": [
            "Women's Clothing"
          ],
          "currency": "EUR",
          "customer_first_name": "Yasmine",
          "customer_full_name": "Yasmine Pratt",
          "customer_gender": "FEMALE",
          "customer_id": 43,
          "customer_last_name": 

### Query 11: Range Query

**Range operators**:
- **gte**: Greater than or equal to
- **gt**: Greater than
- **lte**: Less than or equal to
- **lt**: Less than

In [28]:
result = await query_opensearch(
    """Find orders in the ecommerce index where:
    - Order date is between 2016-01-01 and 2017-12-31
    - Category contains 'Clothing'
    Show me 3 examples"""
)
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...

Invoking: `SearchIndexTool` with `{'index': 'ecommerce', 'query': {'bool': {'must': [{'range': {'order_date': {'gte': '2016-01-01', 'lte': '2017-12-31'}}}, {'match': {'category': 'Clothing'}}]}}, 'size': 3}`


Error searching index: RequestError(400, 'parsing_exception', 'Unknown key for a START_OBJECT in [bool].')
Invoking: `SearchIndexTool` with `{'index': 'ecommerce', 'query': {'bool': {'must': [{'range': {'order_date': {'gte': '2016-01-01', 'lte': '2017-12-31'}}}, {'match_phrase': {'category': 'Clothing'}}]}}, 'size': 3}`
responded: It seems there was an error with the query format. Let me correct it and try again.

Error searching index: RequestError(400, 'parsing_exception', 'Unknown key for a START_OBJECT in [bool].')
Invoking: `SearchIndexTool` with `{'index': 'ecommerce', 'query': {'bool': {'must': [{'range': {'order_date': {'gte': '2016-01-01', 'lte': '2017-12-31'}}}, {'match': {'category': 'Clothing'}}]}}, 'size': 3}`
responded: It se

### Query 12: Aggregation Query

**Aggregations** perform analytics on data:
- **terms**: Group by field values (like SQL GROUP BY)
- **avg/sum/min/max**: Numeric calculations
- **date_histogram**: Time-based grouping

In [29]:
result = await query_opensearch(
    """Analyze the ecommerce index:
    - Count orders by gender
    - Calculate average total amount per gender
    - Show top 5 categories by order count"""
)
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...

Invoking: `ListIndexTool` with `{'include_detail': True}`


All indices information:
[
  {
    "health": "green",
    "status": "open",
    "index": ".plugins-ml-config",
    "uuid": "a9SaiEfNTQOhFOTqseM2vg",
    "pri": "1",
    "rep": "1",
    "docs.count": "1",
    "docs.deleted": "0",
    "store.size": "8.1kb",
    "pri.store.size": "4kb"
  },
  {
    "health": "green",
    "status": "open",
    "index": "top_queries-2025.11.11-85601",
    "uuid": "XE5xrXJaSl6OLJqClKkkng",
    "pri": "1",
    "rep": "1",
    "docs.count": "13",
    "docs.deleted": "0",
    "store.size": "76.5kb",
    "pri.store.size": "38.1kb"
  },
  {
    "health": "green",
    "status": "open",
    "index": "ecommerce",
    "uuid": "okZX79mpQNmJ5TOH0R59Hw",
    "pri": "1",
    "rep": "1",
    "docs.count": "9350",
    "docs.deleted": "0",
    "store.size": "10.6mb",
    "pri.store.size": "5.3mb"
  },
  {
    "health": "green",
    "status": "open",
    "index": "security-au

## 8. Cluster Health and Statistics

```mermaid
graph LR
    A[Cluster Monitoring] --> B[Health Status]
    A --> C[Statistics]
    A --> D[Shard Info]
    
    B --> E[Green: All good<br/>Yellow: Missing replicas<br/>Red: Missing data]
    C --> F[Node count<br/>Document count<br/>Storage size]
    D --> G[Shard allocation<br/>Primary/Replica status]
    
    style A fill:#e1f5ff
    style B fill:#c8e6c9
    style C fill:#fff9e6
    style D fill:#e8f5e9
```

### Cluster Health Colors
- **Green**: All shards allocated, cluster fully operational
- **Yellow**: All primary shards allocated, but some replicas missing
- **Red**: Some primary shards unallocated, data loss possible

### Query 13: Get Cluster Health

In [30]:
result = await query_opensearch("What is the health status of the OpenSearch cluster?")
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...

Invoking: `ClusterHealthTool` with `{}`


{"cluster_name": "opensearch-cluster", "status": "green", "timed_out": false, "number_of_nodes": 2, "number_of_data_nodes": 2, "discovered_master": true, "discovered_cluster_manager": true, "active_primary_shards": 7, "active_shards": 14, "relocating_shards": 0, "initializing_shards": 0, "unassigned_shards": 0, "delayed_unassigned_shards": 0, "number_of_pending_tasks": 0, "number_of_in_flight_fetch": 0, "task_max_waiting_in_queue_millis": 0, "active_shards_percent_as_number": 100.0}The health status of the OpenSearch cluster is "green," indicating that all primary and replica shards are active. Here are some additional details about the cluster:

- Cluster Name: opensearch-cluster
- Number of Nodes: 2
- Number of Data Nodes: 2
- Active Primary Shards: 7
- Active Shards: 14
- Relocating Shards: 0
- Initializing Shards: 0
- Unassigned Shards: 0
- Delayed Unassigned Shards: 0
- Number of Pending Tasks: 0
- 

### Query 14: Get Cluster Statistics

In [31]:
result = await query_opensearch(
    """Show me cluster statistics including:
    - Number of nodes
    - Total number of documents
    - Total storage size
    - Number of indices"""
)
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...

Invoking: `ClusterHealthTool` with `{}`


{"cluster_name": "opensearch-cluster", "status": "green", "timed_out": false, "number_of_nodes": 2, "number_of_data_nodes": 2, "discovered_master": true, "discovered_cluster_manager": true, "active_primary_shards": 7, "active_shards": 14, "relocating_shards": 0, "initializing_shards": 0, "unassigned_shards": 0, "delayed_unassigned_shards": 0, "number_of_pending_tasks": 0, "number_of_in_flight_fetch": 0, "task_max_waiting_in_queue_millis": 0, "active_shards_percent_as_number": 100.0}
Invoking: `ListIndexTool` with `{'include_detail': True}`


All indices information:
[
  {
    "health": "green",
    "status": "open",
    "index": ".plugins-ml-config",
    "uuid": "a9SaiEfNTQOhFOTqseM2vg",
    "pri": "1",
    "rep": "1",
    "docs.count": "1",
    "docs.deleted": "0",
    "store.size": "8.1kb",
    "pri.store.size": "4kb"
  },
  {
    "health": "green",
    "status": "open",
    "index": "top_queries-2025.

## 9. Alias Management

### What are Aliases?
**Aliases** are alternative names for indices that provide:
- **Zero-downtime** index switching
- **Simplified** application code
- **Multiple indices** under one name

Common use case: Blue-green deployments

### Query 15: Create an Alias

In [32]:
result = await query_opensearch("Create an alias 'orders' that points to the 'ecommerce' index")
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...
Creating an alias in OpenSearch typically involves using the OpenSearch API to update the index settings. However, I don't have direct access to execute such operations. You would typically use a command like the following with a tool like `curl` or through a client library:

```bash
curl -X POST "localhost:9200/_aliases" -H 'Content-Type: application/json' -d'
{
  "actions": [
    {
      "add": {
        "index": "ecommerce",
        "alias": "orders"
      }
    }
  ]
}
'
```

This command sends a request to the OpenSearch server to create an alias named "orders" that points to the "ecommerce" index. Make sure to replace `localhost:9200` with your OpenSearch server's address.

> Finished chain.

RESULT:
Creating an alias in OpenSearch typically involves using the OpenSearch API to update the index settings. However, I don't have direct access to execute such operations. You would typically use a command like the following with a tool like `cur

### Query 16: List All Aliases

In [8]:
result = await query_opensearch("Show me all aliases in the cluster and which indices they point to")
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)

/tmp/ipykernel_9100/282870703.py:24: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...

Invoking: `ListIndexTool` with `{'include_detail': True}`


All indices information:
[
  {
    "health": "green",
    "status": "open",
    "index": ".plugins-ml-config",
    "uuid": "Ps5q-MSPTceeVq0_YP4BxA",
    "pri": "1",
    "rep": "1",
    "docs.count": "1",
    "docs.deleted": "0",
    "store.size": "8kb",
    "pri.store.size": "3.9kb"
  },
  {
    "health": "green",
    "status": "open",
    "index": "security-auditlog-2025.11.11",
    "uuid": "a5vTUhz1SSepKYvatHXliw",
    "pri": "1",
    "rep": "1",
    "docs.count": "4",
    "docs.deleted": "0",
    "store.size": "143kb",
    "pri.store.size": "71.5kb"
  },
  {
    "health": "green",
    "status": "open",
    "index": "ecommerce",
    "uuid": "jMvqZKLUQgeunoMv0vEYDw",
    "pri": "1",
    "rep": "1",
    "docs.count": "9350",
    "docs.deleted": "0",
    "store.size": "10.6mb",
    "pri.store.size": "5.3mb"
  },
  {
    "health": "green",
    "status": "open",
    "index": ".kibana_1",
 

### Query 17: Get Alias for Specific Index

In [9]:
result = await query_opensearch("What aliases are configured for the 'ecommerce' index?")
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...

Invoking: `ListIndexTool` with `{'index': 'ecommerce', 'include_detail': True}`


Index information for ecommerce:
{
  "ecommerce": {
    "aliases": {},
    "mappings": {
      "properties": {
        "category": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword"
            }
          }
        },
        "currency": {
          "type": "keyword"
        },
        "customer_birth_date": {
          "type": "date"
        },
        "customer_first_name": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256
            }
          }
        },
        "customer_full_name": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256
            }
          }
        },
        "customer_gender": {
          "type": "keywo

## 10. Data Stream Operations

### What are Data Streams?
**Data streams** are specialized indices for time-series data:
- **Append-only**: Optimized for continuously generated data
- **Automatic rollover**: Creates new backing indices based on conditions
- **Index lifecycle**: Manages data retention and storage tiers

Use cases: Logs, metrics, events, sensor data

### Query 18: Create a Data Stream

Note: Data streams require an index template to exist first.

In [11]:
# First, we need to create an index template using general API
result = await query_opensearch(
    """Create an index template for data streams:
    - Template name: 'logs-template'
    - Index pattern: 'logs-*'
    - Enable data stream
    - Add timestamp field with date type"""
)
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...
To create an index template for data streams in OpenSearch, you need to define the template with the specified settings and mappings. Here's how you can define the template:

```json
{
  "index_patterns": ["logs-*"],
  "data_stream": {},
  "template": {
    "mappings": {
      "properties": {
        "@timestamp": {
          "type": "date"
        }
      }
    }
  }
}
```

This JSON configuration does the following:
- Sets the index pattern to `logs-*`, which matches any index that starts with `logs-`.
- Enables the data stream by including the `"data_stream": {}` field.
- Adds a `@timestamp` field with the `date` type in the mappings.

You can use this JSON configuration to create the index template in OpenSearch. If you need further assistance with applying this template, please let me know!

> Finished chain.

RESULT:
To create an index template for data streams in OpenSearch, you need to define the template with the specified settings and m

### Query 19: List Data Streams

In [10]:
result = await query_opensearch("List all data streams in the cluster")
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...

Invoking: `ListIndexTool` with `{'include_detail': True}`


All indices information:
[
  {
    "health": "green",
    "status": "open",
    "index": ".plugins-ml-config",
    "uuid": "Ps5q-MSPTceeVq0_YP4BxA",
    "pri": "1",
    "rep": "1",
    "docs.count": "1",
    "docs.deleted": "0",
    "store.size": "8kb",
    "pri.store.size": "3.9kb"
  },
  {
    "health": "green",
    "status": "open",
    "index": "security-auditlog-2025.11.11",
    "uuid": "a5vTUhz1SSepKYvatHXliw",
    "pri": "1",
    "rep": "1",
    "docs.count": "10",
    "docs.deleted": "0",
    "store.size": "318kb",
    "pri.store.size": "159kb"
  },
  {
    "health": "green",
    "status": "open",
    "index": "ecommerce",
    "uuid": "jMvqZKLUQgeunoMv0vEYDw",
    "pri": "1",
    "rep": "1",
    "docs.count": "9350",
    "docs.deleted": "0",
    "store.size": "10.6mb",
    "pri.store.size": "5.3mb"
  },
  {
    "health": "green",
    "status": "open",
    "index": ".kibana_1",
 

## 11. General API Request Tool

### Power User Tool
The **general_api_request** tool allows you to make any OpenSearch API call not covered by specific tools.

**HTTP Methods**:
- **GET**: Retrieve data
- **POST**: Create/update data
- **PUT**: Create/update data (idempotent)
- **DELETE**: Remove data

### Query 20: Use General API for Cat API

In [12]:
result = await query_opensearch(
    """Use the cat API to show:
    - All indices with their document count and size
    - Format: human-readable
    - Sort by document count descending"""
)
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...

Invoking: `ListIndexTool` with `{'include_detail': True}`


All indices information:
[
  {
    "health": "green",
    "status": "open",
    "index": ".plugins-ml-config",
    "uuid": "Ps5q-MSPTceeVq0_YP4BxA",
    "pri": "1",
    "rep": "1",
    "docs.count": "1",
    "docs.deleted": "0",
    "store.size": "8kb",
    "pri.store.size": "3.9kb"
  },
  {
    "health": "green",
    "status": "open",
    "index": "security-auditlog-2025.11.11",
    "uuid": "a5vTUhz1SSepKYvatHXliw",
    "pri": "1",
    "rep": "1",
    "docs.count": "10",
    "docs.deleted": "0",
    "store.size": "318kb",
    "pri.store.size": "159kb"
  },
  {
    "health": "green",
    "status": "open",
    "index": "ecommerce",
    "uuid": "jMvqZKLUQgeunoMv0vEYDw",
    "pri": "1",
    "rep": "1",
    "docs.count": "9350",
    "docs.deleted": "0",
    "store.size": "10.6mb",
    "pri.store.size": "5.3mb"
  },
  {
    "health": "green",
    "status": "open",
    "index": ".kibana_1",
 

### Query 21: Update Index Settings

In [13]:
result = await query_opensearch(
    """Update the test_products index settings:
    - Set number_of_replicas to 1
    - Set refresh_interval to 30s"""
)
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...

Invoking: `IndexMappingTool` with `{'index': 'test_products'}`


Error getting mapping: NotFoundError(404, 'index_not_found_exception', 'no such index [test_products]', test_products, index_or_alias)It seems that the `test_products` index does not exist. Would you like to create it with the specified settings, or do you want to check for another index?

> Finished chain.

RESULT:
It seems that the `test_products` index does not exist. Would you like to create it with the specified settings, or do you want to check for another index?


## 12. Advanced Search Scenarios

### Query 22: Multi-field Search

In [14]:
result = await query_opensearch(
    """Search the ecommerce index for:
    - Customer first name or last name containing 'Smith' OR 'Johnson'
    - Order date in the last 6 months of 2024
    - Sort by total amount descending
    - Return top 5 results with customer name and total amount"""
)
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...

Invoking: `SearchIndexTool` with `{'index': 'ecommerce', 'query': {'size': 5, 'sort': [{'total_amount': {'order': 'desc'}}], 'query': {'bool': {'must': [{'range': {'order_date': {'gte': '2024-04-01', 'lte': '2024-10-01'}}}], 'should': [{'match': {'customer_first_name': 'Smith'}}, {'match': {'customer_last_name': 'Smith'}}, {'match': {'customer_first_name': 'Johnson'}}, {'match': {'customer_last_name': 'Johnson'}}]}}, '_source': ['customer_first_name', 'customer_last_name', 'total_amount']}}`


Error searching index: RequestError(400, 'search_phase_execution_exception', 'No mapping found for [total_amount] in order to sort on')
Invoking: `IndexMappingTool` with `{'index': 'ecommerce'}`


Mapping for ecommerce:
{
  "ecommerce": {
    "mappings": {
      "properties": {
        "category": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword"
            }
          }
        },
        "currency

### Query 23: Wildcard Search

In [15]:
result = await query_opensearch(
    """Find all products in the ecommerce index where:
    - Product name starts with 'Shirt'
    - Show the product names and prices
    - Limit to 10 results"""
)
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...

Invoking: `SearchIndexTool` with `{'index': 'ecommerce', 'query': {'size': 10, '_source': ['product_name', 'price'], 'query': {'wildcard': {'product_name.keyword': 'Shirt*'}}}}`


Search results from ecommerce:
{
  "took": 1,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 0,
      "relation": "eq"
    },
    "max_score": null,
    "hits": []
  }
}There are no products in the "ecommerce" index where the product name starts with "Shirt".

> Finished chain.

RESULT:
There are no products in the "ecommerce" index where the product name starts with "Shirt".


### Query 24: Nested Aggregations

In [16]:
result = await query_opensearch(
    """Analyze the ecommerce index:
    - Group by category
    - For each category, show:
      * Total number of orders
      * Average order amount
      * Maximum order amount
    - Sort categories by total order count
    - Show top 5 categories"""
)
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...

Invoking: `ListIndexTool` with `{'index': 'ecommerce', 'include_detail': True}`


Index information for ecommerce:
{
  "ecommerce": {
    "aliases": {},
    "mappings": {
      "properties": {
        "category": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword"
            }
          }
        },
        "currency": {
          "type": "keyword"
        },
        "customer_birth_date": {
          "type": "date"
        },
        "customer_first_name": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256
            }
          }
        },
        "customer_full_name": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256
            }
          }
        },
        "customer_gender": {
          "type": "keywo

## 13. Complex Business Queries

```mermaid
graph TB
    A[Business Question] --> B{LLM Reasoning}
    B --> C[Query Planning]
    C --> D[Tool Selection]
    D --> E[Execute Search]
    E --> F[Process Results]
    F --> G[Format Answer]
    G --> H[Natural Language Response]
    
    style A fill:#e1f5ff
    style B fill:#fff9e6
    style E fill:#e8f5e9
    style H fill:#c8e6c9
```

### Query 25: Revenue Analysis

In [17]:
result = await query_opensearch(
    """What is the total revenue generated from the ecommerce index:
    - In the year 2024
    - From female customers
    - In the 'Men\\'s Clothing' category
    - Also show the average order value"""
)
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...

Invoking: `ListIndexTool` with `{'index': 'ecommerce', 'include_detail': True}`


Index information for ecommerce:
{
  "ecommerce": {
    "aliases": {},
    "mappings": {
      "properties": {
        "category": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword"
            }
          }
        },
        "currency": {
          "type": "keyword"
        },
        "customer_birth_date": {
          "type": "date"
        },
        "customer_first_name": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256
            }
          }
        },
        "customer_full_name": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256
            }
          }
        },
        "customer_gender": {
          "type": "keywo

### Query 26: Customer Segmentation

In [18]:
result = await query_opensearch(
    """Identify high-value customers in the ecommerce index:
    - Customers who have total purchases over 500
    - Group by customer_id
    - Show customer name and total spent
    - List top 10 customers"""
)
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...

Invoking: `ListIndexTool` with `{'include_detail': True}`


All indices information:
[
  {
    "health": "green",
    "status": "open",
    "index": ".plugins-ml-config",
    "uuid": "Ps5q-MSPTceeVq0_YP4BxA",
    "pri": "1",
    "rep": "1",
    "docs.count": "1",
    "docs.deleted": "0",
    "store.size": "8kb",
    "pri.store.size": "3.9kb"
  },
  {
    "health": "green",
    "status": "open",
    "index": "top_queries-2025.11.11-85601",
    "uuid": "hHD0UX7HTACBpgioaQBSiA",
    "pri": "1",
    "rep": "1",
    "docs.count": "12",
    "docs.deleted": "24",
    "store.size": "496.9kb",
    "pri.store.size": "248.1kb"
  },
  {
    "health": "green",
    "status": "open",
    "index": "security-auditlog-2025.11.11",
    "uuid": "a5vTUhz1SSepKYvatHXliw",
    "pri": "1",
    "rep": "1",
    "docs.count": "14",
    "docs.deleted": "0",
    "store.size": "153.9kb",
    "pri.store.size": "76.9kb"
  },
  {
    "health": "green",
    "status": "open",
   

### Query 27: Time-based Analysis

In [19]:
result = await query_opensearch(
    """Analyze order patterns in the ecommerce index:
    - Group orders by month in 2024
    - For each month show:
      * Number of orders
      * Total revenue
    - Which month had the highest revenue?"""
)
print("\n" + "="*80)
print("RESULT:")
print("="*80)
print(result)



> Entering new AgentExecutor chain...

Invoking: `ListIndexTool` with `{'include_detail': True}`


All indices information:
[
  {
    "health": "green",
    "status": "open",
    "index": ".plugins-ml-config",
    "uuid": "Ps5q-MSPTceeVq0_YP4BxA",
    "pri": "1",
    "rep": "1",
    "docs.count": "1",
    "docs.deleted": "0",
    "store.size": "8kb",
    "pri.store.size": "3.9kb"
  },
  {
    "health": "green",
    "status": "open",
    "index": "top_queries-2025.11.11-85601",
    "uuid": "hHD0UX7HTACBpgioaQBSiA",
    "pri": "1",
    "rep": "1",
    "docs.count": "12",
    "docs.deleted": "24",
    "store.size": "496.9kb",
    "pri.store.size": "248.1kb"
  },
  {
    "health": "green",
    "status": "open",
    "index": "security-auditlog-2025.11.11",
    "uuid": "a5vTUhz1SSepKYvatHXliw",
    "pri": "1",
    "rep": "1",
    "docs.count": "15",
    "docs.deleted": "0",
    "store.size": "182.9kb",
    "pri.store.size": "91.4kb"
  },
  {
    "health": "green",
    "status": "open",
   

## 14. Summary of MCP Tools

### All Available Tools

| Tool Name | Category | Description | Example Use |
|-----------|----------|-------------|-------------|
| **list_indices** | Index Mgmt | List all indices | Inventory check |
| **get_index** | Index Mgmt | Get index details | View mappings/settings |
| **create_index** | Index Mgmt | Create new index | Setup new data store |
| **delete_index** | Index Mgmt | Delete index | Cleanup old data |
| **index_document** | Document Ops | Add/update document | Insert new record |
| **get_document** | Document Ops | Retrieve by ID | Fetch specific record |
| **delete_document** | Document Ops | Remove by ID | Delete single record |
| **delete_by_query** | Document Ops | Remove by criteria | Bulk deletion |
| **search_documents** | Search | Query documents | Find matching data |
| **get_cluster_health** | Cluster | Check health status | Monitor cluster |
| **get_cluster_stats** | Cluster | Get statistics | View metrics |
| **list_aliases** | Alias | List all aliases | View mappings |
| **get_alias** | Alias | Get index aliases | Check specific index |
| **put_alias** | Alias | Create/update alias | Setup alias |
| **delete_alias** | Alias | Remove alias | Cleanup alias |
| **create_data_stream** | Data Streams | Create stream | Time-series setup |
| **get_data_stream** | Data Streams | Get stream info | View stream details |
| **delete_data_stream** | Data Streams | Delete stream | Cleanup streams |
| **general_api_request** | Advanced | Custom API call | Any OpenSearch API |

### Key Learnings

1. **MCP bridges the gap** between AI agents and data sources
2. **Natural language queries** are translated to precise API calls
3. **LangChain + MCP** enables conversational data access
4. **GPT-4 reasoning** selects the right tools automatically
5. **Streaming transport** enables real-time programmatic access

## 15. Cleanup

Let's clean up the resources we created during this demonstration.

In [20]:
# Delete test index
try:
    if client.indices.exists(index='test_products'):
        client.indices.delete(index='test_products')
        print("✓ Deleted test_products index")
except Exception as e:
    print(f"Note: {e}")

# Delete alias
try:
    if client.indices.exists_alias(name='orders'):
        client.indices.delete_alias(index='ecommerce', name='orders')
        print("✓ Deleted orders alias")
except Exception as e:
    print(f"Note: {e}")

print("\nCleanup completed!")


Cleanup completed!


In [21]:
# Stop MCP server
try:
    mcp_process.terminate()
    mcp_process.wait(timeout=5)
    print("✓ MCP server stopped successfully")
except:
    mcp_process.kill()
    print("✓ MCP server forcefully stopped")

✓ MCP server stopped successfully


## 16. Next Steps and Resources

### What You've Learned
✅ MCP architecture and components  
✅ Installing and configuring opensearch-mcp-server  
✅ LangChain integration with OpenAI  
✅ All 18+ MCP tools for OpenSearch  
✅ Natural language to OpenSearch query translation  
✅ Complex queries and aggregations  
✅ Cluster management and monitoring  

### Advanced Topics to Explore
1. **Multi-cluster mode**: Connect to multiple OpenSearch clusters
2. **IAM authentication**: Use AWS IAM roles for security
3. **Tool filtering**: Enable/disable specific tools
4. **Custom tool customization**: Modify tool descriptions and parameters
5. **Production deployment**: Run MCP server as a service

### Resources
- [OpenSearch MCP Server GitHub](https://github.com/opensearch-project/opensearch-mcp-server-py)
- [Model Context Protocol Specification](https://modelcontextprotocol.io/)
- [LangChain Documentation](https://python.langchain.com/)
- [OpenSearch Documentation](https://opensearch.org/docs/)

### Practice Exercises
1. Create a multi-tenant search system using aliases
2. Build a log analysis pipeline with data streams
3. Implement a recommendation system using aggregations
4. Design an anomaly detection workflow
5. Create a customer 360 view with complex queries

---

## Congratulations! 🎉

You've completed the comprehensive OpenSearch MCP Server demonstration. You now understand how to bridge AI agents with OpenSearch using the Model Context Protocol, enabling natural language interfaces for complex data operations.

**Key Takeaway**: MCP + LangChain + OpenSearch = Powerful conversational data access! 🚀